In [ ]:
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

In [1]:
import pandas as pd
import numpy as np
from features.preprocess_catboost import preprocess_project_data as load_data
from utils.metrics import regression_metrics, quantile_coverage, print_metrics
from models.linear_model import load_features_and_split, train_linear_regression
from models.catboost_model import train_catboost, train_catboost_quantiles, predict_interval as cat_predict_interval
from models.lightgbm_model import train_lightgbm, train_lightgbm_quantiles, predict_interval as lgbm_predict_interval
from models.random_forest import train_random_forest
from models.xgboost_model import train_xgboost

ModuleNotFoundError: No module named 'features'

In [4]:
from sklearn.model_selection import train_test_split

In [9]:
import linear_model
print(linear_model.__file__)

c:\Users\Utilisateur\Desktop\KUL\S2\ModernDataAnalytics\Test\Modern-Data-Analytics\models\linear_model.py


In [ ]:

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "models")))




ImportError: cannot import name 'load_features_and_split' from 'models.linear_model' (c:\Users\Utilisateur\Desktop\KUL\S2\ModernDataAnalytics\Test\Modern-Data-Analytics\models\linear_model.py)

In [6]:
# Add the *parent* of models to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, os.path.join(project_root, "models"))


In [6]:
cat_features = [col for col in X_train.columns if X_train[col].dtype == "object"]
print("Categorical feature names:", cat_features)


Categorical feature names: ['fundingScheme', 'masterCall']


In [7]:
catboost_params = {
    "iterations": 800,
    "learning_rate": 0.03,
    "depth": 6,
    "verbose": 0,
    "random_seed": 32
}

catboost_quantile_params = {
    "iterations": 500,
    "learning_rate": 0.03,
    "depth": 6,
    "verbose": 0,
    "random_seed": 42
}

lgbm_params = {
    "n_estimators": 500,
    "learning_rate": 0.03,
    "num_leaves": 31,
    "verbose": -1
}

lgbm_quantile_params = {
    "n_estimators": 500,
    "learning_rate": 0.03,
    "num_leaves": 31,
    "verbose": -1
}

rf_params = {"n_estimators": 200, "max_depth": 6, "random_state": 42}
linreg_params = {}  # scikit-learn LinearRegression has no meaningful hyperparameters
xgb_params = {"n_estimators": 500, "learning_rate": 0.03, "max_depth": 6, "verbosity": 0, "random_state": 42}

In [10]:
cat_model = train_catboost(X_train, y_train,catboost_params=catboost_params,cat_features=cat_features)

y_pred_cat = cat_model.predict(X_test)
print_metrics("CatBoost Point", regression_metrics(y_test, y_pred_cat))

cat_low = train_catboost_quantiles(X_train, y_train, alpha=0.05, catboost_quantile_params=catboost_quantile_params, cat_features=cat_features)
cat_med = train_catboost_quantiles(X_train, y_train, alpha=0.5, catboost_quantile_params=catboost_quantile_params, cat_features=cat_features)
cat_high = train_catboost_quantiles(X_train, y_train, alpha=0.95, catboost_quantile_params=catboost_quantile_params, cat_features=cat_features)
intervals_cat = cat_predict_interval(cat_low, cat_med, cat_high, X_test)
print_metrics("CatBoost Interval", quantile_coverage(y_test, intervals_cat))


CatBoost Point Metrics:
RMSE: 89.1130
MAE: 64.2151
R2: 0.4931

CatBoost Interval Metrics:
Interval Coverage: 0.8912
Avg Interval Width: 266.8153


In [ ]:
lgbm_model = train_lightgbm(X_train, y_train, lgbm_params=lgbm_params)
y_pred_lgbm = lgbm_model.predict(X_test)
print_metrics("LightGBM Point", regression_metrics(y_test, y_pred_lgbm))

lgbm_low = train_lightgbm_quantiles(X_train, y_train, alpha=0.05, lgbm_quantile_params=lgbm_quantile_params)
lgbm_med = train_lightgbm_quantiles(X_train, y_train, alpha=0.5, lgbm_quantile_params=lgbm_quantile_params)
lgbm_high = train_lightgbm_quantiles(X_train, y_train, alpha=0.95, lgbm_quantile_params=lgbm_quantile_params)
intervals_lgbm = lgbm_predict_interval(lgbm_low, lgbm_med, lgbm_high, X_test)
print_metrics("LightGBM Interval", quantile_coverage(y_test, intervals_lgbm))

In [ ]:
rf_model = train_random_forest(X_train, y_train, rf_params=rf_params)
y_pred_rf = rf_model.predict(X_test)
print_metrics("Random Forest", regression_metrics(y_test, y_pred_rf))

In [ ]:
def print_metrics(model_name, metrics_dict):
    print(f"\n📊 Performance for {model_name}")
    for k, v in metrics_dict.items():
        print(f"{k}: {v:.4f}")

# Load data
X_train, X_test, y_train, y_test = load_features_and_split()

# Train model
linreg_model = train_linear_regression(X_train, y_train)

# Predict
y_pred_lin = linreg_model.predict(X_test)

# Evaluate
metrics = regression_metrics(y_test, y_pred_lin)
print_metrics("Linear Regression", metrics)

In [ ]:
xgb_model = train_xgboost(X_train, y_train, xgb_params=xgb_params)
y_pred_xgb = xgb_model.predict(X_test)
print_metrics("XGBoost", regression_metrics(y_test, y_pred_xgb))